In [12]:
import json
import sys
import os

In [ ]:
def find_files(base_dir, begin_date, end_date):
    
    search_dirs = []
    
    stop_date = begin_date

    while stop_date < (end_date + relativedelta(months = +1)):
        search_dirs.append(stop_date.strftime('%Y/%Y_%m'))

        stop_date += relativedelta(months = +1)

    filenames=[]

    for search_dir in search_dirs:
        filenames += sorted(glob.glob(os.path.join(base_dir, search_dir, '*.nc')))
        
    return filenames

In [7]:
filename = "/home/will/Projects/headliner/headline-store-json/the-new-york-times-2019-12-26T12:00:00-p001.json"

with open(filename, "r") as file:
    data = json.load(file)

dict_keys(['status', 'totalResults', 'articles'])